# STATISTICAL BASELINE
## We want to classify mightee sources using the mahalanobis distance (simple stats)

### Code adopted from Eslam

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sources.stats_f1 import*

from sklearn.model_selection import train_test_split

In [ ]:
# We call the test and train data saved from the processing notebook

mightee_data = pd.read_csv('raw_data.csv')

X_data = pd.read_csv('X_train.csv')
y_data = pd.read_csv('y_train.csv')

y = y_data['labels']

ml_feat = [['qir'], 
           ['qir', 'class_star'],
           ['qir', 'class_star', 'log(S8/S45)'],
           ['qir', 'class_star', 'log(S8/S45)','log(S58/S36)'],
           ['qir', 'class_star', 'log(S8/S45)','log(S58/S36)', 'Mstar'],
           ['qir', 'class_star', 'log(S8/S45)','log(S58/S36)', 'Mstar', 'log(S45/S36)']]

In [ ]:
splits = [0.2]
base_dict = {} # https://www.geeksforgeeks.org/python-nested-dictionary/  (for more exploration around dictionaries) 



In [ ]:
for s in splits:
    X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size= s, random_state=1, stratify = y, shuffle = True)
    
    i = 1
    for f in ml_feat:
        xtr =  X_train[f]
        xte =  X_test[f]
        results = get_f1_base (xtr, y_train, xte, y_test, len(f)) # to get the f1scores for the baseline
        
        key = "F"+str((i))
        
        base_dict[key] = {}
        base_dict[key]['tot_f1'] = results[0]
        base_dict[key]['jack_train'] = results[1]
        base_dict[key]['jack_test'] = results[2]
        i += 1

In [ ]:
%store base_dict 
# we will use this Dictionary in the next notebook

In [ ]:
print(base_dict.keys())
key = list(base_dict.keys())[0]
print( base_dict[key].keys() )

In [ ]:
f1_arr = [] 
sd_arr = [] 


for key in base_dict.keys():
    
    print( str(key)+": "+str(base_dict[key][ 'tot_f1' ] ))
    f1_arr.append(base_dict[key][ 'tot_f1' ]) # append total f1 to an array
    sd_train = jack_SD( np.zeros( len(base_dict[key][ 'jack_train' ]) ), base_dict[key][ 'jack_train' ])[0]
    sd_test = jack_SD( np.zeros( len(base_dict[key][ 'jack_test' ]) ), base_dict[key][ 'jack_test' ])[0]
    sd = np.sqrt( np.array((sd_train**2)) + np.array((sd_test**2)))
    sd_arr.append(sd) # append sd_arr to an array

    print(sd)
    print()
    

In [ ]:
plt.figure(figsize=(10 , 7))
plt.title( "BASELINE F1 scores for different features with SD", fontweight ='bold', fontsize =16)
plt.xlabel("Features", fontweight ='bold', fontsize =12)
plt.xticks(rotation=45, ha='right')
plt.ylim(0.75, 1)
plt.ylabel("F1 Scores", fontweight ='bold', fontsize =12)
plt.errorbar( list(base_dict.keys()), f1_arr, sd_arr, fmt='o')
plt.show()